In [1]:
import tensorflow as tf
import numpy as np

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Read MNIST data
from tensorflow.examples.tutorials.mnist import input_data
dataset = input_data.read_data_sets("data/MNIST/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/MNIST/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
# Placeholders for MNIST images
img_holder = tf.placeholder(tf.float32, [None, 784])
lbl_holder = tf.placeholder(tf.float32, [None, 10])
train = tf.placeholder(tf.bool)
# Layer settings
hid_nodes = 200
out_nodes = 10
keep_prob = 0.5

In [4]:
from tensorflow.contrib import layers
from tensorflow.contrib.layers import fully_connected
# Create layers
with tf.contrib.framework.arg_scope(
    [fully_connected],
    normalizer_fn=tf.contrib.layers.batch_norm,
    normalizer_params={'is_training': train}):
        layer1 = fully_connected(img_holder, hid_nodes, scope='layer1')
        layer1_drop = tf.layers.dropout(layer1, keep_prob, training=train)
        layer2 = fully_connected(layer1_drop, hid_nodes, scope='layer2')
        layer2_drop = tf.layers.dropout(layer2, keep_prob, training=train)
        layer3 = fully_connected(layer2_drop, hid_nodes, scope='layer3')
        layer3_drop = tf.layers.dropout(layer3, keep_prob, training=train)
        out_layer = fully_connected(layer3_drop, out_nodes, activation_fn=None, scope='layer4')


In [6]:
print(np.shape(layer1))
print(np.shape(layer1_drop))
print(np.shape(layer2))
print(np.shape(layer2_drop))
print(np.shape(layer3))
print(np.shape(layer3_drop))
print(np.shape(out_layer))

(?, 200)
(?, 200)
(?, 200)
(?, 200)
(?, 200)
(?, 200)
(?, 10)


In [5]:
# Compute loss
loss = tf.reduce_mean(
tf.nn.softmax_cross_entropy_with_logits(
logits=out_layer, labels=lbl_holder))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [18]:
# Create optimizer
learning_rate = 0.01
num_epochs = 50
batch_size = 100
num_batches = int(dataset.train.num_examples/batch_size)
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [19]:
# Initialize variables
init = tf.global_variables_initializer()

In [20]:
sess = tf.Session()
sess.run(init)
for epoch in range(num_epochs):
    for patch in range(num_batches):
        img_batch, lbl_batch = dataset.train.next_batch(batch_size)
        loss_, _ = sess.run([loss, optimizer], 
        feed_dict = {img_holder: img_batch, lbl_holder: lbl_batch, train: True})
print(loss_)
# Determine success rate
prediction = tf.equal(tf.argmax(out_layer, 1), tf.argmax(lbl_holder, 1))
success = tf.reduce_mean(tf.cast(prediction, tf.float32))
print('Success rate: ', sess.run(success,
feed_dict={img_holder: dataset.test.images,
lbl_holder: dataset.test.labels, train: False}))

0.46000713
Success rate:  0.9514
